In [1]:
import sys;sys.path.append('..')
from ppanlp import *
import networkx as nx
ppa = PPA()
def get_work_id(pageid):
    return '.'.join(pageid.split('.')[:-1])

2023-12-07 13:02:08,976 | booting PPACorpus at /Users/.../ppa_data/corpus
2023-12-07 13:02:08,977 | | reading metadata
2023-12-07 13:02:09,097 | | 0.12 seconds
2023-12-07 13:02:09,150 | 0.17 seconds


In [3]:
df=pd.read_csv(os.path.join(ppa.path_data, 'data.ner+topic.stats.v5.csv'))

In [17]:
df[(df.type_x=='Person') & (df.type_y=='Person')][['entity_x','entity_y','num_total_x','num_total_y','prob_x','prob_y','prob_xy_exp','prob_xy_obs','prob_xy_obsexp']]

,entity_x,entity_y,num_total_x,num_total_y,prob_x,prob_y,prob_xy_exp,prob_xy_obs,prob_xy_obsexp
1,Rustum,Sohrab,111,126,0.045817,0.052008,0.000024,0.035910,150703.582154
6,Anselm,Lanfranc,134,108,0.055310,0.044579,0.000025,0.024353,98769.147319
10,Juliet,Romeo,946,844,0.390475,0.348373,0.001360,0.264582,19450.120362
11,Absalom,Achitophel,351,164,0.144880,0.067693,0.000098,0.047468,48399.928775
14,Beowulf,Hygelac,689,100,0.284395,0.041276,0.000117,0.034672,29536.423803
...,...,...,...,...,...,...,...,...,...
406419,Aristotle,Malory,4047,218,1.670457,0.089983,0.001503,0.000000,0.000000
406420,Aristotle,Chevalier,4047,101,1.670457,0.041689,0.000696,0.000000,0.000000
406421,Aristotle,Thopas,4047,219,1.670457,0.090395,0.001510,0.000000,0.000000
406422,Aristotle,Strauss,4047,105,1.670457,0.043340,0.000724,0.000000,0.000000


In [55]:
def iter_counts(words,min_doc_len=25,max_per_cluster=100):
    words={w.lower() for w in words} | {' '}
    for page in ppa.iter_pages(use_db=True,as_dict=True,min_doc_len=min_doc_len,max_per_cluster=max_per_cluster):
        # pagewords=set(tokenize_agnostic(page['page_text'].lower()))
        # for match in words&pagewords:
        ptext = page['page_text'].lower()
        for word in words:
            if word in ptext:
                wd={'word':word, 'page_id':page['page_id'], 'work_id': get_work_id(page['page_id']), 'num_pages':1}
                yield wd

In [93]:
@cache
def count_by_period(words, period_len=10, **kwargs):
    df = pd.DataFrame(iter_counts(words,**kwargs))
    dfc = df.groupby(['work_id','word']).sum(numeric_only=True).sort_values('num_pages',ascending=False)
    dfall = dfc.reset_index().set_index('work_id').join(ppa.meta)
    dfall['year']=pd.to_numeric(dfall.pub_date,errors='coerce')
    dfall['period']=dfall.year//period_len*period_len
    dfprd=dfall[dfall.year>=1700].groupby(['period','word'])[['num_pages']].sum(numeric_only=True).reset_index()
    dfprd_sums=dfprd.query('word==" "').set_index('period').num_pages
    dfprd['num_pages_rel'] = [npg/dfprd_sums.loc[prd]*100 for npg,prd in zip(dfprd.num_pages,dfprd.period)]
    dfprd=dfprd[dfprd.word!=' ']
    return dfprd

In [59]:
words = {'ballad','sonnet','lyric','line'}

In [60]:
dfprd = count_by_period(words, min_doc_len=25, max_per_cluster=100)

2023-12-07 12:02:45,822 | iterating pages by page database
2023-12-07 12:02:45,827 | | iterating page database results ({"min_doc_len": 25, "frac_min": 0, "frac_max": 1})
Iterating over page search results: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2044105/2044105 [01:34<00:00, 21692.92it/s]
2023-12-07 12:04:20,060 | | 1 minute and 34.23 seconds
2023-12-07 12:04:20,060 | 1 minute and 34.24 seconds


In [62]:
dfprd

,period,word,num_pages,num_pages_rel
1,1700.0,line,274,15.163254
2,1700.0,lyric,28,1.549530
3,1700.0,sonnet,6,0.332042
5,1710.0,ballad,11,0.343428
6,1710.0,line,565,17.639713
...,...,...,...,...
108,1910.0,sonnet,955,1.852570
110,1920.0,ballad,652,2.919708
111,1920.0,line,6196,27.746182
112,1920.0,lyric,1057,4.733330


In [87]:
# !pip install altair
import altair as alt
fig1a=alt.Chart(dfprd).mark_line().encode(
    x='period:Q',
    y='num_pages_rel:Q',
    color='word:N',
)

alt.Chart(...)

In [88]:
alt.Chart(dfprd[dfprd.word!='line']).mark_line().encode(
    x='period:Q',
    y='num_pages_rel:Q',
    color='word:N',
)

alt.Chart(...)

In [86]:
fig3a=alt.Chart(dfprd).mark_line().encode(
    x='period:Q',
    y='num_pages:Q',
    color='word:N',
)

alt.Chart(...)

In [85]:
dfprd2 = count_by_period(words, min_doc_len=25, max_per_cluster=None)
dfprd2

2023-12-07 12:10:32,609 | iterating pages by page database
2023-12-07 12:10:32,616 | | iterating page database results ({"min_doc_len": 25, "frac_min": 0, "frac_max": 1})
Iterating over page search results:   8%|████████████████▍                                                                                                                                                                                          | 165928/2044105 [00:07<01:28, 21215.39it/s]
2023-12-07 12:10:40,441 | | 7.82 seconds
2023-12-07 12:10:40,442 | 7.83 seconds


KeyboardInterrupt: 

In [80]:
# !pip install altair
import altair as alt
fig1b=alt.Chart(dfprd2).mark_line().encode(
    x='period:Q',
    y='num_pages_rel:Q',
    color='word:N',
)
fig2b=alt.Chart(dfprd2[dfprd2.word!='line']).mark_line().encode(
    x='period:Q',
    y='num_pages_rel:Q',
    color='word:N',
)
fig3b=alt.Chart(dfprd2).mark_line().encode(
    x='period:Q',
    y='num_pages:Q',
    color='word:N',
)

In [81]:
display(fig1a)
display(fig1b)

alt.Chart(...)

alt.Chart(...)

In [82]:
display(fig2a)
display(fig2b)

alt.Chart(...)

alt.Chart(...)

In [84]:
display(fig3a)
display(fig3b)

alt.Chart(...)

alt.Chart(...)

In [99]:
words = ('milton','chaucer','shakespeare','wordsworth','browning')
data = count_by_period(words, min_doc_len=25, max_per_cluster=100, period_len=10)
fig = alt.Chart(data).mark_line().encode(
    x='period',
    y='num_pages_rel',
    color='word',
).properties(
    width=800,
    height=500
)
display(fig)

alt.Chart(...)